In [1]:
import qiskit
from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiments.library import ProcessTomography

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis

# Noisy simulator backend
backend = AerSimulator.from_backend(FakeParis())


/var/folders/49/5xjhcql1577_5wbzp23ykxk40000gq/T/ipykernel_96539/3489064930.py:7: DeprecationWarning: The module 'qiskit.test.mock' is deprecated since Qiskit Terra 0.21.0, and will be removed 3 months or more later. Instead, you should import the desired object directly 'qiskit.providers.fake_provider'.
  from qiskit.test.mock import FakeParis
/Users/lior/miniconda3/lib/python3.9/site-packages/qiskit/providers/aer/noise/device/models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (78724.73839768364 > 2 * 17977.586136949183). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
/Users/lior/miniconda3/lib/python3.9/site-packages/qiskit/providers/aer/noise/device/models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (151392.40575779957 > 2 * 69749.41874968237). Truncating to maximum v

In [2]:
# Run experiments

# GHZ State preparation circuit
nq = 2
qc_ghz = qiskit.QuantumCircuit(nq)
qc_ghz.h(0)
qc_ghz.s(0)
for i in range(1, nq):
    qc_ghz.cx(0, i)

# QST Experiment
qstexp1 = ProcessTomography(qc_ghz)
qstdata1 = qstexp1.run(backend, seed_simulation=100).block_for_results()


In [ ]:
qstdata1

In [54]:
for circ in qstexp1.circuits()[::9]:
    print(circ.draw())


          ┌─────────────┐ ░ ┌───┐┌───┐      ░ ┌────────────┐ ░ ┌─┐   
q_0: ─|0>─┤ PauliPrepZp ├─░─┤ H ├┤ S ├──■───░─┤ PauliMeasZ ├─░─┤M├───
          ├─────────────┤ ░ └───┘└───┘┌─┴─┐ ░ ├────────────┤ ░ └╥┘┌─┐
q_1: ─|0>─┤ PauliPrepZp ├─░───────────┤ X ├─░─┤ PauliMeasZ ├─░──╫─┤M├
          └─────────────┘ ░           └───┘ ░ └────────────┘ ░  ║ └╥┘
c: 2/═══════════════════════════════════════════════════════════╩══╩═
                                                                0  1 
          ┌─────────────┐ ░ ┌───┐┌───┐      ░ ┌────────────┐ ░ ┌─┐   
q_0: ─|0>─┤ PauliPrepZp ├─░─┤ H ├┤ S ├──■───░─┤ PauliMeasZ ├─░─┤M├───
          ├─────────────┤ ░ └───┘└───┘┌─┴─┐ ░ ├────────────┤ ░ └╥┘┌─┐
q_1: ─|0>─┤ PauliPrepZm ├─░───────────┤ X ├─░─┤ PauliMeasZ ├─░──╫─┤M├
          └─────────────┘ ░           └───┘ ░ └────────────┘ ░  ║ └╥┘
c: 2/═══════════════════════════════════════════════════════════╩══╩═
                                                                0  1 
          ┌─────────

In [16]:
pms = [inst for inst in iter(circ) if 'PauliMeas' in inst.name]

AttributeError: 'CircuitInstruction' object has no attribute 'name'

In [20]:
c.operation.

'measure'

In [65]:
from qm.qua import *
from typing import Callable
from itertools import product


def initialize_qubits(qubits):
    pass


def measure_q(q):
    measure('readout', q)


def pauli_meas(op, qubit):
    if op == 'Z':
        play('id', qubit)
    elif op == 'X':
        play('-y/2', qubit)  # verify correctness of pulse
    elif op == 'Y':
        play('x/2', qubit)  # verify correctness of pulse


def pauli_prep(op, qubit):
    if op == 'Zp':
        play('id', qubit)
    elif op == 'Zm':
        play('x', qubit)
    elif op == 'Xp':
        play('-y/2', qubit)  # verify correctness of pulse
    elif op == 'Yp':
        play('x/2', qubit)  # verify correctness of pulse


def state_tomography_wrapper(qubits, circ: Callable, *circ_args, **circ_kwargs):
    pauli_meas_cases = list(product(['Z', 'X', 'Y'], repeat=len(qubits)))
    meas_case_i = declare(int)
    with for_(meas_case_i, 0, meas_case_i < len(pauli_meas_cases), meas_case_i+1):

        # state preparation
        initialize_qubits(qubits)

        # circuit
        circ(*circ_args, **circ_kwargs)

        # measure
        with switch_(meas_case_i, unsafe=True):
            for i, case in enumerate(pauli_meas_cases):
                with case_(i):
                    for op, q in zip(case, qubits):
                        pauli_meas(op, q)
        for q in qubits:
            measure_q(q)


def my_circ():
    play('x', 'q1')


def process_tomography_wrapper(qubits, circ: Callable, *circ_args, **circ_kwargs):

    pauli_prep_cases = list(
        product(['Zm', 'Zp', 'Xp', 'Yp'], repeat=len(qubits)))
    prep_case_i = declare(int)

    pauli_meas_cases = list(product(['Z', 'X', 'Y'], repeat=len(qubits)))
    meas_case_i = declare(int)

    with for_(prep_case_i, 0, prep_case_i < len(pauli_prep_cases), prep_case_i+1):
        with for_(meas_case_i, 0, meas_case_i < len(pauli_meas_cases), meas_case_i+1):

            # state preparation
            initialize_qubits(qubits)

            with switch_(prep_case_i):
                for i, case in enumerate(pauli_prep_cases):
                    with case_(i):
                        for op, q in zip(case, qubits):
                            pauli_prep(op, q)

            # circuit
            circ(*circ_args, **circ_kwargs)

            # measure
            with switch_(meas_case_i, unsafe=True):
                for i, case in enumerate(pauli_meas_cases):
                    with case_(i):
                        for op, q in zip(case, qubits):
                            pauli_meas(op, q)
            for q in qubits:
                measure_q(q)


In [58]:
with program() as state_prog:
    state_tomography_wrapper(['0', '1'], my_circ)

from qm import generate_qua_script

print(generate_qua_script(state_prog))



# Single QUA script generated at 2022-07-11 20:38:36.083957
# QUA library version: 0.3.7

from qm.qua import *

with program() as prog:
    v1 = declare(int, )
    with for_(v1,0,v1<9,v1+1):
        play("x", "q1")
        with if_(v1==0, unsafe=True):
            wait(20, "0")
            wait(20, "1")
        with elif_(v1==1):
            wait(20, "0")
            play("-y/2", "1")
        with elif_(v1==2):
            wait(20, "0")
            play("x/2", "1")
        with elif_(v1==3):
            play("-y/2", "0")
            wait(20, "1")
        with elif_(v1==4):
            play("-y/2", "0")
            play("-y/2", "1")
        with elif_(v1==5):
            play("-y/2", "0")
            play("x/2", "1")
        with elif_(v1==6):
            play("x/2", "0")
            wait(20, "1")
        with elif_(v1==7):
            play("x/2", "0")
            play("-y/2", "1")
        with elif_(v1==8):
            play("x/2", "0")
            play("x/2", "1")
        measure("rea

In [67]:
with program() as process_prog:
    process_tomography_wrapper(['0', '1'], my_circ)

from qm import generate_qua_script

print(generate_qua_script(process_prog))



# Single QUA script generated at 2022-07-11 20:49:53.508369
# QUA library version: 0.3.7

from qm.qua import *

with program() as prog:
    v1 = declare(int, )
    v2 = declare(int, )
    with for_(v1,0,v1<16,v1+1):
        with for_(v2,0,v2<9,v2+1):
            with if_(v1==0):
                pass
            with elif_(v1==1):
                play("id", "1")
            with elif_(v1==2):
                play("-y/2", "1")
            with elif_(v1==3):
                play("x/2", "1")
            with elif_(v1==4):
                play("id", "0")
            with elif_(v1==5):
                play("id", "0")
                play("id", "1")
            with elif_(v1==6):
                play("id", "0")
                play("-y/2", "1")
            with elif_(v1==7):
                play("id", "0")
                play("x/2", "1")
            with elif_(v1==8):
                play("-y/2", "0")
            with elif_(v1==9):
                play("-y/2", "0")
                play("id"